# ncRNA prediction pipeline

For ncRNA prediction 3 programs were utilized:

- Infernal: Used to predict multiple types of ncRNA comparing against the Rfam database
- tRNAscan: Used to precit eukaryotic tRNAs
- RNAmmer: Used to predict RNA sub

## Input:

1. Sacha inchi draft genome : "Sacha_softmasked.fasta"
2. Sacha inchi Maker gene prediciton **(for tRNAscan only)**: "Sacha_genepred.gff"
3. Rfam database covariance models (cm) **(for Infernal only)**

<br><br><br>
# Infernal

Flowchart:<br>
<img src="./images/ncRNA_analisis.png" width=450>

## Input:

1. Sacha inchi draft genome : "Sacha_softmasked.fasta"
3. Rfam database covariance models (cm)


First precalibrated cms of all Rfam families are downloaded from the Rfam v14.5 FTP site, then extracted and merged to be used on infernal

In [1]:
!mkdir rfam_cms
!cd rfam_cms

!wget ftp://ftp.ebi.ac.uk/pub/databases/Rfam/14.5/Rfam.tar.gz
!tar -xvf Rfam.tar.gz & rm Rfam.tar.gz

#All the cm files are concatenated in a file, "all_families.cm"
!cat *.cm > all_families.cm

!cd ..
!ls -lh rfam_cms/

total 8,0K
drwxr-xr-x 2 anderjackf anderjackf 4,0K abr  5 22:36 images
-rw-r--r-- 1 anderjackf anderjackf 2,2K abr  5 22:50 Sacha_ncRNA_prediction.ipynb


After de Rfam data is set up we can run infernal to get predictions:

In [ ]:
!genome="genome/Sacha_softmasked.fasta"
!database="rfam_cms/all_families.cm"
!mkdir infernal_output
!output="infernal_output/"

!cmscan --tblout $output/result_infernal.tbl --fmt 2 --notextw --cpu 24 $database $genome

Knowing that the 19th column stores data about the significance of the hits, any record that has a "!" is a significant hit. An E-vale of 0.01 is considered significant. In total 7131 hits were significant.

In [ ]:
!awk -v OFS="\t" '{ if ($19 == "!") print $0}' result_infernal.tbl > significant_hits.tsv

Downloading entry type by family data on Rfam(https://rfam.xfam.org/search#tabview=tab5) we can clasify hits in broad categories. To do this we'll use a python helper script .

In [8]:
!python infernal_pie_chart.py RFAM_entry_type.tsv significant_hits.tsv ./images/pie_charts

The graphs:<br>
<img src="./images/pie_charts_1.png" width=800><br>
<img src="./images/pie_charts_2.png" width=800>

As you can see most of the hits correspond to Genes, the rest are Introns ans Cis-reg elements. Within the "Genes" category we see that most of the hits correspond to miRNA rRNA snRNA and tRNA. 

<br><br><br>
# tRNAscan

For this prediction tRNAscan was run within MAKER gene prediction pipeline, so the results of tRNAscan prediction are withtin the sacha gff file prediction. To extract the tRNA type occurrence data, the following command is used:

In [10]:
!grep "trnascan" /home/anderjackf/Universidad/Semillero_biocomp/Sacha/experimental/gene_prediction/backup/run_20/gene_models_manual_work_2.gff | egrep -o "trnascan.*noncoding-.*-gene" | cut -f 4 -d "-" | python item_occurrence.py
#!grep "trnascan" Sacha_genepred.gff | egrep -o "trnascan.*noncoding-.*-gene" | cut -f 4 -d "-" | python item_occurrence.py

Pseudo_Gln	1	0.001869158878504673
Pseudo_Gly	1	0.001869158878504673
Pseudo_Asn	1	0.001869158878504673
Pseudo_Arg	1	0.001869158878504673
Pseudo_Phe	1	0.001869158878504673
Pseudo_iMet	1	0.001869158878504673
Pseudo_Glu	1	0.001869158878504673
Pseudo_Lys	1	0.001869158878504673
Pseudo_Leu	2	0.003738317757009346
Pseudo_Ser	2	0.003738317757009346
Sup	2	0.003738317757009346
Pseudo_Asp	3	0.005607476635514018
Undet	3	0.005607476635514018
Pseudo_Val	4	0.007476635514018692
Pseudo_Undet	7	0.013084112149532711
iMet	10	0.018691588785046728
Cys	12	0.022429906542056073
Trp	13	0.024299065420560748
His	14	0.026168224299065422
Tyr	16	0.029906542056074768
Asn	16	0.029906542056074768
Phe	17	0.03177570093457944
Ile	18	0.03364485981308411
Gln	18	0.03364485981308411
Asp	24	0.044859813084112146
Lys	24	0.044859813084112146
Thr	24	0.044859813084112146
Met	25	0.04672897196261682
Pro	28	0.052336448598130844
Val	28	0.052336448598130844
Glu	29	0.05420560747663551
Gly	33	0.0616822429906542
Ala	33	0.0616822429906542
Arg

In [8]:
#!grep "trnascan" /home/anderjackf/Universidad/Semillero_biocomp/Sacha/experimental/gene_prediction/backup/run_20/gene_models_manual_work_2.gff | egrep -o "trnascan.*noncoding-.*-gene" | cut -f 4 -d "-" | python codon_occurrence.py
!grep "trnascan" Sacha_genepred.gff | egrep -o "trnascan.*noncoding-.*-gene" | cut -f 4 -d "-" | python codon_occurrence.py

Ile:
 	TAT	6	0.3333333333333333
 	AAT	12	0.6666666666666666
-----------------------------------------
Pseudo_Undet:
 	NNN	7	1.0
-----------------------------------------
His:
 	GTG	14	1.0
-----------------------------------------
Val:
 	GAC	2	0.07142857142857142
 	TAC	5	0.17857142857142858
 	CAC	7	0.25
 	AAC	14	0.5
-----------------------------------------
Thr:
 	GGT	1	0.041666666666666664
 	CGT	3	0.125
 	TGT	10	0.4166666666666667
 	AGT	10	0.4166666666666667
-----------------------------------------
Ser:
 	CGA	3	0.06818181818181818
 	GGA	6	0.13636363636363635
 	TGA	9	0.20454545454545456
 	AGA	10	0.22727272727272727
 	GCT	16	0.36363636363636365
-----------------------------------------
Lys:
 	CTT	11	0.4583333333333333
 	TTT	13	0.5416666666666666
-----------------------------------------
Pseudo_Arg:
 	TCT	1	1.0
-----------------------------------------
Leu:
 	CAG	5	0.11363636363636363
 	TAG	7	0.1590909090909091
 	TAA	10	0.22727272727272727
 	CAA	11	0.25
 	AAG	11	0.25
--------------------

In the previous result, the second column corresponds to tRNA type counts, and the third corresponds to tRNA type
frecuency. Leucine, Serine and Arginine are the most common tRNAs found on Sacha inchi's draft genome

<br><br><br>
# RNAmmer